In [ ]:
!pip install dash yfinance plotly pandas

In [69]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import yfinance as yf
import pandas as pd
from functools import lru_cache

# Logos dictionary (add logos manually or dynamically via an API)
LOGOS = {
    "ASML": "https://logo.clearbit.com/asml.com",
    "NVO": "https://logo.clearbit.com/novonordisk.com",
    "VALE": "https://logo.clearbit.com/vale.com",
    "BHP": "https://logo.clearbit.com/bhp.com",
    "PBR": "https://logo.clearbit.com/petrobras.com",
    "AMD": "https://logo.clearbit.com/amd.com",
    "NEE": "https://logo.clearbit.com/nee.com",
    "NVDA": "https://logo.clearbit.com/nvidia.com",
    "GOOG": "https://logo.clearbit.com/google.com",
    "ROL": "https://logo.clearbit.com/rollins.com",
    "COIN": "https://logo.clearbit.com/coinbase.com",
    # Add more tickers and corresponding logos here...
}

# Cache data for 5 minutes to avoid frequent data fetching
@lru_cache(maxsize=32)
def get_cached_stock_data():
    tickers = list(LOGOS.keys())  # Only track stocks with available logos for now
    try:
        # Fetch price, volume, and adjusted close for tickers (5 days of data for % change)
        data = yf.download(tickers, period='5d')  # Fetch data for the last 5 days

        # Only process the data if it was downloaded successfully
        if data.empty:
            return pd.DataFrame(), pd.DataFrame()

        # Creating the stock dataframe with required columns
        stock_data = pd.DataFrame(index=tickers)
        stock_data['Price'] = data['Adj Close'].iloc[-1]  # Use the last available close price
        stock_data['Change'] = data['Adj Close'].pct_change().iloc[-1] * 100  # Percentage change from the previous day
        stock_data['Change'] = stock_data['Change'].round(2)
        stock_data['Vol'] = data['Volume'].iloc[-1]

        # Simplified fetching only the necessary fields from info (avoiding repeated API calls)
        info_data = yf.Tickers(tickers).tickers
        stock_data['Avg Vol'] = [info_data[ticker].info.get('averageVolume', 0) for ticker in tickers]
        stock_data['RVol'] = (stock_data['Vol'] / stock_data['Avg Vol']).round(2)
        stock_data['Float'] = [info_data[ticker].info.get('floatShares', 0) for ticker in tickers]
        stock_data['Mcap'] = [info_data[ticker].info.get('marketCap', 0) for ticker in tickers]
        stock_data['Logo'] = [LOGOS.get(ticker, "") for ticker in tickers]

        # Handle missing data (fill missing values with 0 or a default value)
        stock_data = stock_data.fillna({
            'Price': 0,
            'Change': 0,
            'Vol': 0,
            'RVol': 0,
            'Float': 0,
            'Mcap': 0
        })

        # Filtering the data for gainers (positive change) and losers (negative change)
        gainers = stock_data[stock_data['Change'] > 0].sort_values(by='Change', ascending=False).head(10)
        losers = stock_data[stock_data['Change'] < 0].sort_values(by='Change', ascending=True).head(10)

        return gainers, losers
    except Exception as e:
        print(f"Error fetching data: {e}")
        return pd.DataFrame(), pd.DataFrame()

# Function to format large numbers with K, M, or B
def format_large_number(num):
    if num >= 1e9:
        return f"{num/1e9:.2f}B"
    elif num >= 1e6:
        return f"{num/1e6:.2f}M"
    elif num >= 1e3:
        return f"{num/1e3:.2f}K"
    else:
        return f"{num:.0f}"

# Function to generate HTML table without a header for the "Logo" column and with center alignment
def generate_table(dataframe):
    # Ensure the dataframe has data
    if dataframe.empty:
        return html.Div("No data available", style={'color': 'white', 'text-align': 'center'})

    # Create the table with formatted numbers, no header for the "Logo" column, and center alignment
    return html.Table([
        html.Thead(
            html.Tr([html.Th('', style={'text-align': 'center'}),  # Empty header for the logo column
                     html.Th('Ticker', style={'text-align': 'center'}),
                     html.Th('Price', style={'text-align': 'center'}),
                     html.Th('Change (%)', style={'text-align': 'center'}),
                     html.Th('Vol', style={'text-align': 'center'}),
                     html.Th('RVol', style={'text-align': 'center'}),
                     html.Th('Float', style={'text-align': 'center'}),
                     html.Th('Mcap', style={'text-align': 'center'})])
        ),
        html.Tbody([
            html.Tr([
                html.Td(html.Img(src=dataframe['Logo'][i], style={'width': '30px'} if dataframe['Logo'][i] else {'width': '30px', 'background-color': 'gray'}), style={'text-align': 'center'}),
                html.Td(dataframe.index[i], style={'text-align': 'center'}),
                html.Td(f"{dataframe['Price'][i]:.2f}", style={'text-align': 'center'}),  # Price with two decimals
                html.Td(
                    f"{dataframe['Change'][i]:.2f}%",
                    style={'color': 'green', 'text-align': 'center'} if dataframe['Change'][i] > 0 else {'color': 'red', 'text-align': 'center'}
                ),  # Change with color: green for gainers, red for losers
                html.Td(format_large_number(dataframe['Vol'][i]), style={'text-align': 'center'}),  # Format volume with M or K
                html.Td(f"{dataframe['RVol'][i]:.2f}", style={'text-align': 'center'}),  # Relative volume with two decimals
                html.Td(format_large_number(dataframe['Float'][i]), style={'text-align': 'center'}),  # Format float with M
                html.Td(format_large_number(dataframe['Mcap'][i]), style={'text-align': 'center'})  # Format market cap with B
            ]) for i in range(len(dataframe))
        ])
    ], style={'width': '100%', 'border-collapse': 'collapse', 'color': 'white', 'background-color': 'black'})

# Create the app
app = dash.Dash(__name__)

# Layout of the dashboard
app.layout = html.Div([
    html.H1("Active Stock", style={'text-align': 'center', 'color': 'black'}),
    html.Div([
        html.Div([
            html.H3("Biggest Gainers", style={'color': 'green', 'background-color': 'white', 'text-align': 'center'}),
            html.Div(id='gainers-table')
        ], style={'width': '45%', 'display': 'inline-block', 'background-color': 'black', 'color': 'white'}),

        html.Div([
            html.H3("Biggest Losers", style={'color': 'red', 'background-color': 'white', 'text-align': 'center'}),
            html.Div(id='losers-table')
        ], style={'width': '45%', 'display': 'inline-block', 'float': 'right', 'background-color': 'black', 'color': 'white'})
    ]),

    # Interval to update the data every 5 minutes
    dcc.Interval(id='interval-component', interval=5*60000, n_intervals=0)
], style={'background-color': 'white', 'padding': '10px'})

# Callback to update the gainers and losers data
@app.callback(
    [Output('gainers-table', 'children'), Output('losers-table', 'children')],
    [Input('interval-component', 'n_intervals')]
)
def update_tables(n):
    gainers, losers = get_cached_stock_data()

    # If data is unavailable, return empty figures
    if gainers.empty or losers.empty:
        return html.Div(), html.Div()

    return generate_table(gainers), generate_table(losers)

# Run the server
if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>